## Notebook description/goals

    -Learning the MOCSY toolbox for calc. carbonate chemistry. 
    -Development of pipeline for MOCSY-based calculation of air-sea co2 flux
    -examples https://github.com/jamesorr/mocsy/blob/master/notebooks/mocsy_examples.ipynb
    
### steps

    -import model output and make arrays from: temp, sal, alk, dic, sil (?) phos? 
    
    -figure out how to get wind (do i need to interpolate it?)
    
    -write down equation for air-sea flux of CO2 and perform sanity check. 
    
    -eventually: put this into an automatic script that:
        -retrieves model output
        -retreives wind
        -calculates air-sea flux of co2 and o2 and puts them into a netcdf (probably along with wind speed)

In [1]:
%reset 

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
import sys
#sys.path.append('/data/tjarniko/MEOPAR/mocsy')
sys.path.append('/data/tjarniko/mocsy')
sys.path.append('/data/tjarniko/MEOPAR/at3/notebooks/carbon_dev/VIZ')
import mocsy
import VIZ_plots as VP

import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from oct2py import octave
import seawater as sw
from __future__ import print_function
from numpy import *
from scipy import *
import matplotlib.pyplot as plt
import seawater as sw

import matplotlib.pyplot as plt
from matplotlib import reload
plt.style.use('seaborn-bright')
import netCDF4 as nc
import numpy as np
import cmocean as cm

""
from salishsea_tools import (
    nc_tools,
    viz_tools,
    geo_tools,
    tidetools
)

%matplotlib inline

## load dataset

In [3]:
print(mocsy)

<module 'mocsy' from '/data/tjarniko/mocsy/mocsy.cpython-36m-x86_64-linux-gnu.so'>


In [4]:
infil = loadtxt('../MODEL_EVAL/tereza.txt')

In [5]:

crid= infil[:,0]
ln = infil[:,2]
stn = infil[:,3]
mon = infil[:,4]
day = infil[:,5]
lat_or = infil[:,6]
lon_or = infil[:,7]
# lat = infil_cor[:,6]
# lon = infil_cor[:,7]
P = infil[:,8]
T = infil[:,9]
S = infil[:,10]
ox = infil[:,11]
ox_qf = infil[:,12]
dic = infil[:,13]
alk = infil[:,15]
dic_qf = infil[:,14]
alk_qf = infil[:,16]
no3 = infil[:,17]
no3_qf = infil[:,18]
si = infil[:,19]
si_qf = infil[:, 20]
po4 = infil[:,21]
po4_qf = infil[:, 22]



In [6]:
dens = sw.dens(S,T,P)
dic_conv=dic *1e-6
alk_conv=alk *1e-6
si_conv = si*1e-6
po4_conv = po4*1e-6

In [7]:
print(dens)
print(dic)
print(dic_conv)

[ 1024.10773013  1025.76483947  1026.86751858]
[ 1995.42  2162.43  2234.39]
[ 0.00199542  0.00216243  0.00223439]


In [8]:
PATM = np.zeros_like(dic)
PATM[:] = 10.0
print(PATM)

[ 10.  10.  10.]


In [9]:
pH,pco2,fco2,co2,hco3,co3,OmegaA,OmegaC,BetaD,DENis,p,Tis \
    = mocsy.mvars(temp=T , sal=S , alk=alk_conv, dic=dic_conv, sil=si_conv, phos=po4_conv, 
                            patm=PATM, depth=P, lat=lat_or, optcon='mol/kg', optt='Tinsitu', optp='db',
                            optb = 'l10', optk1k2='m10', optkf = 'dg', optgas = 'Pinsitu' )

In [10]:
print(pH,pco2,fco2,co2,hco3,co3,OmegaA,OmegaC,BetaD,DENis,p,Tis)

[ 8.0913865   7.76458247  7.63150298] [  368.52097551   856.39270226  1226.43618246] [  354.66266175   814.25078649  1131.42757907] [  1.36428925e-05   3.68089484e-05   5.34128074e-05] [ 0.00182909  0.00205796  0.00213158] [  1.52687243e-04   6.76576339e-05   4.93996498e-05] [ 2.35936768  1.02629738  0.73592221] [ 3.69907876  1.62149087  1.16311237] [ 11.41077978  17.26132801  18.40156289] [ 1024.10774199  1025.7648567   1026.86753655] [   4.9   29.3  102.8] [ 14.195   8.94    7.258]


###### HERE IS THE CO2SYS OUTPUT to compare
##### pH is on the TOtal scale - PCO2 (uatm) - balk (umol/kg) - PH (umol/kg)
#
#z [  4.9  29.3 102.8]
#dic [1995.42 2162.43 2234.39]
#pco2 [ 346.43071199  795.59072662 1096.50343874]
#pH [8.09712784 7.76907771 7.63518989]
#omega_arag [2.3880103  1.03658638 0.74203948]
#borate alk [71.04906401 33.37843368 24.81288429]
#OH- (oh) [2.62367006 0.73844744 0.46316284]

In [11]:
print('pco2 is:')
print(pco2)

pco2 is:
[  368.52097551   856.39270226  1226.43618246]


In [12]:
print('aragonite is:')
print(OmegaA)

aragonite is:
[ 2.35936768  1.02629738  0.73592221]


In [13]:
print('pH is:')
print(pH)

pH is:
[ 8.0913865   7.76458247  7.63150298]


In [14]:
# pH,pco2,fco2,co2,hco3,co3,OmegaA,OmegaC,BetaD,DENis,p,Tis = \
#     mocsy.mvars(temp=15, sal=32, alk=2100, dic=2100, sil=0, phos=0, 
#                 patm=1, depth=0, lat=0, 
#                 optcon='mol/kg', optt='Tinsitu', optp='db', 
#                 optb="u74", optk1k2='l', optkf="dg", optgas='Pinsitu')
# print(pH,pco2,fco2,co2,hco3,co3,OmegaA,OmegaC,BetaD,DENis,p,Tis)
# print('**')
# print(co2)